<a href="https://colab.research.google.com/github/MithunSrinivas28/Fluxo_AI_Supplychain/blob/main/demand_forecasting_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🚀 PHASE 2 — MODEL BUILDING

# STEP 1 — Load the Dataset

In [46]:
import pandas as pd
import numpy as np

df = pd.read_csv("synthetic_supplychain_data.csv")
df.head()

,product_id,product_name,category,zone,warehouse,year,week,month,is_festival,base_demand,base_price,current_price,discount_percent,zone_multiplier,warehouse_multiplier,year_growth,units_sold_next_week
0,1,Fertilizer,agriculture,North,A,1.0,1.0,1.0,0.0,627,278,271.024431,15,1.05,0.959,1.0,875
1,1,Fertilizer,agriculture,North,A,1.0,2.0,1.0,0.0,627,278,258.874636,0,1.05,0.959,1.0,738
2,1,Fertilizer,agriculture,North,A,1.0,3.0,1.0,0.0,627,278,253.429449,10,1.05,0.959,1.0,903
3,1,Fertilizer,agriculture,North,A,1.0,4.0,1.0,0.0,627,278,251.344498,15,1.05,0.959,1.0,934
4,1,Fertilizer,agriculture,North,A,1.0,5.0,2.0,0.0,627,278,296.483811,0,1.05,0.959,1.0,577


In [47]:
df.columns

Index(['product_id', 'product_name', 'category', 'zone', 'warehouse', 'year',
       'week', 'month', 'is_festival', 'base_demand', 'base_price',
       'current_price', 'discount_percent', 'zone_multiplier',
       'warehouse_multiplier', 'year_growth', 'units_sold_next_week'],
      dtype='object')

In [48]:
df = df.sort_values(
    by=["product_id", "zone", "warehouse", "year", "week"]
)

df["lag_1"] = df.groupby(
    ["product_id", "zone", "warehouse"]
)["units_sold_next_week"].shift(1)

df["rolling_4_avg"] = df.groupby(
    ["product_id", "zone", "warehouse"]
)["units_sold_next_week"].rolling(4).mean().reset_index(level=[0,1,2], drop=True)

df = df.dropna()

df[["lag_1", "rolling_4_avg"]].head()

,lag_1,rolling_4_avg
939,988.0,787.25
940,744.0,760.75
941,658.0,767.50
942,680.0,765.25
943,979.0,757.00


In [49]:
X = df[[
    "current_price",
    "base_price",
    "discount_percent",
    "year_growth",
    "month",
    "is_festival",
    "product_id",
    "zone",
    "warehouse",
    "category",
    "lag_1",
    "rolling_4_avg",
    "year"
]]

y = df["units_sold_next_week"]

In [50]:
df.columns

Index(['product_id', 'product_name', 'category', 'zone', 'warehouse', 'year',
       'week', 'month', 'is_festival', 'base_demand', 'base_price',
       'current_price', 'discount_percent', 'zone_multiplier',
       'warehouse_multiplier', 'year_growth', 'units_sold_next_week', 'lag_1',
       'rolling_4_avg'],
      dtype='object')

In [51]:
# Select features
X = df[[
    "current_price",
    "base_price",
    "discount_percent",
    "year_growth",
    "month",
    "is_festival",
    "product_id",
    "zone",
    "warehouse",
    "category",
    "lag_1",
    "year"
]]

y = df["units_sold_next_week"]

Time-Based Split

In [52]:
train = X[X["year"] < 3]
test = X[X["year"] == 3]

y_train = y[X["year"] < 3]
y_test = y[X["year"] == 3]

train = train.drop(columns=["year"])
test = test.drop(columns=["year"])

One-Hot Encode

In [53]:
train = pd.get_dummies(
    train,
    columns=["zone", "warehouse", "category"],
    drop_first=True
)

test = pd.get_dummies(
    test,
    columns=["zone", "warehouse", "category"],
    drop_first=True
)

train, test = train.align(test, join="left", axis=1, fill_value=0)

Train Random Forest

In [54]:
from sklearn.ensemble import RandomForestRegressor

rf_model = RandomForestRegressor(
    n_estimators=200,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(train, y_train)

predictions = rf_model.predict(test)

Evaluate

In [55]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

mae = mean_absolute_error(y_test, predictions)
rmse = np.sqrt(mean_squared_error(y_test, predictions))

print("MAE:", mae)
print("RMSE:", rmse)

MAE: 52.98582064636753
RMSE: 83.60661069422785


In [56]:
feature_importance = pd.DataFrame({
    "feature": train.columns,
    "importance": rf_model.feature_importances_
}).sort_values(by="importance", ascending=False)

feature_importance.head(10)

,feature,importance
7,lag_1,0.792809
0,current_price,0.087788
1,base_price,0.030737
2,discount_percent,0.027088
6,product_id,0.013579
14,category_electronics,0.008488
4,month,0.005146
10,zone_West,0.005062
5,is_festival,0.004658
17,category_grains,0.004578
